In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

## I. 関数

### 1. 主たる居室・その他の居室・非居室に接する外皮の部位の面積・長さ

#### 1) 面積

$$ A_{i,j=1} = \frac{ A_{MR} }{ A_A } A_i $$

$$ A_{i,j=2} = \frac{ A_{OR} }{ A_A  } A_i $$

$$A_{i,j=3} = \frac{ A_A - A_{MR} - A_{OR} }{ A_A}   A_i$$

#### 2) 長さ

$$ l_{i,j=1} = \frac{ A_{MR} }{ A_A } l_i $$

$$ l_{i,j=2} = \frac{ A_{OR} }{ A_A } l_i $$

$$ l_{i,j=3} = \frac{ A_A - A_{MR} - A_{OR} }{ A_A } l_i $$

$A_i$ : 外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$A_{i,j}$ : 外皮の部位（一般部位・開口部）$(i,j)$の面積（ただし$j=1$の時は主たる居室に接し、$j=2$の時はその他の居室に接し、$j=3$の時は非居室に接する）, m<sup>2</sup>  
$l_i$ : 外皮の部位（熱橋等）$i$の面積, m  
$l_{i,j}$ : 外皮の部位（熱橋等）$(i,j)$の面積（ただし$j=1$の時は主たる居室に接し、$j=2$の時はその他の居室に接し、$j=3$の時は非居室に接する）, m  
$A_A$ : 床面積の合計, m<sup>2</sup>  
$A_{MR}$ : 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$ : その他の居室の床面積, m<sup>2</sup>  

In [2]:
def get_separated_areas(a_i, a_mr, a_or, a_a):

    a_i_1 = a_mr / a_a * a_i
    a_i_2 = a_or / a_a * a_i
    a_i_3 = (a_a - a_mr - a_or) / a_a * a_i
    
    return a_i_1, a_i_2, a_i_3

In [3]:
def get_separated_lengths(l_i, a_mr, a_or, a_a):

    l_i_1 = a_mr / a_a * l_i
    l_i_2 = a_or / a_a * l_i
    l_i_3 = (a_a - a_mr - a_or) / a_a * l_i
    
    return l_i_1, l_i_2, l_i_3

### 2. 外皮の部位の分類

#### 1) 一般部

In [4]:
def make_general_part(name, evlp_type, next_space, direction, spec, space_type, separated_area, **kwargs ):

    return {
        'name'       : name + '_' + space_type,
        'evlp_type'  : evlp_type,
        'next_space' : next_space,
        'direction'  : direction,
        'area'       : separated_area,
        'space_type' : space_type,
        'spec'       : copy.deepcopy(spec)        
    }

In [5]:
def get_general_parts(general_parts, common):
    
    separated_general_parts =[]
    
    for general_part in general_parts:
        
        a_i_1, a_i_2, a_i_3 = get_separated_areas(
            a_i=general_part['area'], 
            a_mr=common['main_occupant_room_floor_area'],
            a_or=common['other_occupant_room_floor_area'], 
            a_a=common['total_floor_area']
        )
        
        separated_general_parts.append(make_general_part(
            **general_part,
            space_type = 'main_occupant_room',
            separated_area = a_i_1
        ))
        
        separated_general_parts.append(make_general_part(
            **general_part,
            space_type = 'other_occupant_room',
            separated_area = a_i_2
        ))
        
        separated_general_parts.append(make_general_part(
            **general_part, 
            space_type = 'non_occupant_room', 
            separated_area = a_i_3
        ))
        
    return separated_general_parts

example

In [6]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor', 
                'direction'  : 'top', 
                'area'       : 67.8,
                'spec'       : 'something',
            },
            {
                'name'       : 'floor',
                'evlp_type'  : 'floor',
                'next_space' : 'outdoor', 
                'direction'  : 'bottom', 
                'area'       : 67.8,
                'spec'       : 'something',
            },
            {
                'name'       : 'wall',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor', 
                'direction'  : 'n', 
                'area'       : 67.8,
                'spec'       : 'something',
            },
        ],
    }
} 

get_general_parts(d['envelope']['general_parts'], d['common'])

[{'name': 'ceiling_main_occupant_room',
  'evlp_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_other_occupant_room',
  'evlp_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 33.9,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_non_occupant_room',
  'evlp_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'floor_main_occupant_room',
  'evlp_type': 'floor',
  'next_space': 'outdoor',
  'direction': 'bottom',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'floor_other_occupant_room',
  'evlp_type': 'floor',
  'next_space': 'outdoor',
  'direction': 'bottom',
  'area': 33.9,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'floor_non_occupant_ro

#### 3) 大部分がガラスで構成される窓等の開口部

In [7]:
def make_window(name, next_space, direction, spec, space_type, separated_area, **kwargs ):
    
    return {
        'name'       : name + '_' + space_type,
        'nexts_pace' : next_space,
        'direction'  : direction,
        'area'       : separated_area,
        'space_type' : space_type,
        'spec'       : copy.deepcopy(spec)
    }

In [8]:
def get_windows(windows, common):
    
    separated_windows =[]
    
    for window in windows:
        
        a_i_1, a_i_2, a_i_3 = get_separated_areas(
            a_i=window['area'], 
            a_mr=common['main_occupant_room_floor_area'], 
            a_or=common['other_occupant_room_floor_area'], 
            a_a=common['total_floor_area']
        )

        separated_windows.append(make_window(
            **window,
            space_type = 'main_occupant_room',
            separated_area = a_i_1
        ))
                                 
        separated_windows.append(make_window(
            **window,
            space_type = 'other_occupant_room',
            separated_area = a_i_2
        ))
                                 
        separated_windows.append(make_window(
            **window,
            space_type = 'non_occupant_room',
            separated_area = a_i_3
        ))
        
    return separated_windows

example

In [9]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            }
        ]
    }
}    

get_windows(d['envelope']['windows'], d['common'])

[{'name': 'window_sw_main_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_other_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'sw',
  'area': 15.125,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_non_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_main_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_other_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'nw',
  'area': 1.585,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_non_occupant_room',
  'nexts_pace': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'non_occupant_room',
  'sp

#### 4) 大部分がガラスで構成されないドア等の開口部

In [10]:
def make_door(name, next_space, direction, spec, space_type, separated_area, **kwargs ):

    return {
        'name'       : name + '_' + space_type,
        'next_space' : next_space,
        'direction'  : direction,
        'area'       : separated_area,
        'space_type' : space_type,
        'spec'       : copy.deepcopy(spec)
    }

In [11]:
def get_doors(doors, common):
    
    separated_doors = []
    
    for door in doors:
        
        a_i_1, a_i_2, a_i_3 = get_separated_areas(
            a_i=door['area'], 
            a_mr=common['main_occupant_room_floor_area'], 
            a_or=common['other_occupant_room_floor_area'], 
            a_a=common['total_floor_area']
        )
        
        separated_doors.append(make_door(
            **door,
            space_type = 'main_occupant_room',
            separated_area = a_i_1
        ))
        
        separated_doors.append(make_door(
            **door,
            space_type = 'other_occupant_room',
            separated_area = a_i_2
        ))
        
        separated_doors.append(make_door(
            **door,
            space_type = 'non_occupant_room',
            separated_area = a_i_3
        ))
        
    return separated_doors

example

In [12]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                'spec'       : 'something'
            },
            {
                'name'       : 'doorNE',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 2.16,
                'spec'       : 'something'
            }
        ]
    }
}    

get_doors(d['envelope']['doors'], d['common'])

[{'name': 'door_nw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 1.26,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 1.08,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'non_occupant_room',
  'spec': 'something'}]

#### 5) 熱橋

In [13]:
def make_heatbridge(name, next_space, direction, space_type, separated_length, spec, **kwargs ):
    
    return {
        'name'       : name + '_' + space_type,
        'next_space' : next_space,
        'direction'  : direction,
        'space_type' : space_type,
        'length'     : separated_length,
        'spec'       : copy.deepcopy(spec),
    }

In [14]:
def get_heatbridges(heatbridges, common):
    
    separated_heatbridges =[]
    
    for heatbridge in heatbridges:

        l_i_1, l_i_2, l_i_3 = get_separated_lengths(
            l_i=heatbridge['length'], 
            a_mr=common['main_occupant_room_floor_area'], 
            a_or=common['other_occupant_room_floor_area'], 
            a_a=common['total_floor_area']
        )
        
        separated_heatbridges.append(make_heatbridge(
            **heatbridge,
            space_type = 'main_occupant_room',
            separated_length = l_i_1
        ))
        separated_heatbridges.append(make_heatbridge(
            **heatbridge,
            space_type = 'other_occupant_room',
            separated_length = l_i_2
        ))
        separated_heatbridges.append(make_heatbridge(
            **heatbridge,
            space_type = 'non_occupant_room',
            separated_length = l_i_3
        ))
        
    return separated_heatbridges

example

In [15]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                'length'     : 1.0,
                'spec'       : 'something',
            },
            {
                'name'       : 'heatbridge_nw',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'w'],
                'length'     : 2.0,
                'spec'       : 'something',
            }
        ]
    }
}    

get_heatbridges(d['envelope']['heatbridges'], d['common'])

[{'name': 'heatbridge_ne_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'main_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_ne_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'other_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_ne_non_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'non_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_nw_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'main_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_nw_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'other_occupant_room',
  'length': 1.0,
  'spec': 'something'},
 {'name': 'heatbridge_nw_non_occupan

#### 6) 土間床等の外周部

In [16]:
def make_earthfloor_perimeter(name, next_space, direction, length, spec, **kwargs ):
    
    return {
        'name'       : name,
        'next_space' : next_space,
        'direction'  : direction,
        'length'     : length,
        'space_type' : 'underfloor',
        'spec'       : copy.deepcopy(spec),
    }

In [17]:
def get_earthfloorperimeters(earthfloor_perimeters):    
    return [ make_earthfloor_perimeter(**f) for f in earthfloor_perimeters]

example

In [18]:
d = {
    'earthfloor_perimeters': [
        {
            'name'       : 'earthfloor_perimeter_nw',
            'next_space' : 'outdoor',
            'direction'  : 'nw',
            'length'     : 2.43,
            'spec': 'something'
        },
        {
            'name'       : 'earthfloor_perimeter_ne',
            'next_space' : 'outdoor',
            'direction'  : 'ne',
            'length'     : 2.43,
            'spec': 'something'
        }
    ]
}    

get_earthfloorperimeters(d['earthfloor_perimeters'])

[{'name': 'earthfloor_perimeter_nw',
  'next_space': 'outdoor',
  'direction': 'nw',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'},
 {'name': 'earthfloor_perimeter_ne',
  'next_space': 'outdoor',
  'direction': 'ne',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'}]

#### 7) 土間床等の中心部

In [19]:
def make_earthfloor(name, area):
    
    return {
        'name'       : name,
        'area'       : area,
        'space_type' : 'underfloor'
    }

In [20]:
def get_earthfloor_centers(earthfloor_centers):        
    return [ make_earthfloor(**f) for f in earthfloor_centers]

example

In [21]:
d = {
    'earthfloor_centers': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}    

get_earthfloor_centers(d['earthfloor_centers'])

[{'name': 'other', 'area': 5.0, 'space_type': 'underfloor'},
 {'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor'}]

## II. 統合

In [22]:
def convert(d):
    
    common = d['common']
    envelope = d['envelope']
    
    output_envelope = {}
    
    if ('general_parts' in envelope) == True:
        output_envelope['general_parts'] = get_general_parts(envelope['general_parts'], common)
    if ('windows' in envelope) == True:
        output_envelope['windows'] = get_windows(envelope['windows'], common)
    if ('doors' in envelope) == True:
        output_envelope['doors'] = get_doors(envelope['doors'], common)
    if ('heatbridges' in envelope) == True:
        output_envelope['heatbridges'] = get_heatbridges(envelope['heatbridges'], common)
    if ('earthfloor_perimeters' in envelope) == True:
        output_envelope['earthfloor_perimeters'] = get_earthfloorperimeters(envelope['earthfloor_perimeters'])
    if ('earthfloor_centers' in d['envelope']) == True:
        output_envelope['earthfloor_centers'] = get_earthfloor_centers(envelope['earthfloor_centers'])

    return {
        'common'   : copy.deepcopy(common),
        'envelope' : output_envelope,
    }

example

In [23]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputUA',                     
                    'UA': 0.24,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllDetails',
                    'IsInContactWithOutsideAir': True, 
                    'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'floor',
                'evlp_type'  : 'floor',
                'next_space' : 'outdoor',
                'direction'  : 'bottom',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers',
                    'IsInContactWithOutsideAir': True, 
                    'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
                    'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                     {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                     {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'wall',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'n',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
                    'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'RoofInsulRafter', 
                    'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'boundary_ceiling',
                'evlp_type'  : 'boundary_ceiling',
                'next_space' : 'air_conditioned',
                'direction'  : 'upward',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
                    'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                       {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                       {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False
                }
            },
            
                        {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood', 
                    'InputMethod': 'InputUR', 
                    'InsulationMethod': 'CeilingInsulbeam',
                    'IsInContactWithOutsideAir': True,              
                    'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'RC', 
                    'InputMethod': 'InputUA', 
                    'UA': 0.24, 
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'RC',
                    'InputMethod': 'InputLayers', 
                    'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'steel',
                    'InputMethod': 'InputUA', 
                    'UA': 0.24,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'steel',
                    'InputMethod': 'InputUR', 
                    'IsInContactWithOutsideAir': True,
                    'area': 67.8,
                    'Radding': 0.10, 
                    'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'other',
                    'InputMethod': None, 
                    'IsSunshadeInput': False 
                }
            },
        ],
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 6.51,
                    'TypeWindow': 'Single', 
                    'EtaInputMethod': 'InputValue',
                    'TypeFrame': 'WoodOrResin',
                    'TypeGlass': '3WgG',
                    'TypeShade': 'Shoji',
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            },
            {
                'name'       : 'window_nw', 
                'next_space' : 'outdoor', 
                'direction'  : 'nw', 
                'area'       : 3.17, 
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 4.65, 
                    'TypeWindow': 'Single', 
                    'EtaInputMethod': 'InputValue',
                    'Eta': 0.738, 
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 6.51,
                    'TypeWindow' : 'Double', 
                    'EtaInputMethod': 'InputSpecification',
                    'TypeFrameInside': 'WoodOrResin',
                    'TypeGlassInside': '3WgG',
                    'TypeShadeInside': 'Shoji',
                    'TypeFrameOutside': 'Steel', 
                    'TypeGlassOutside': '3WgG', 
                    'TypeShadeOutside': 'ExtarnalBlind',
                    'IsSunshadeInput': True, 
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            },
            { 
                'name'       : 'window_nw',
                'next_space' : 'outdoor', 
                'direction'  : 'nw',
                'area'       : 3.17,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 4.65,
                    'TypeWindow': 'Double', 
                    'EtaInputMethod': 'InputValue',
                    'EtaInside': 0.738,
                    'TypeGlassInside': '3WgG',
                    'EtaOutside': 0.738, 
                    'TypeGlassOutside': '3WgG',
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00, 
                    'Z': 0.60
                }
            }
        ],
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U'               : 6.51,
                    'IsSunshadeInput' : False  
                }
            },
            {
                'name'       : 'door_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 2.16,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            }
        ],
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'length'     : 1.00,
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure' : 'steel',
                    'psi'       : 1.8,
                }
            },
            {
                'name'       : 'heatbridge_nw',
                'length'     : 2.00,
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'w'],
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure' : 'steel',
                    'psi'       : 1.8,
                }
            },
        ],
        'earthfloor_perimeters': [
            {
                'name'       : 'earthfloor_perimeter_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'length'     : 2.43,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'length'     : 2.43,
                'spec'       : {
                    'psi': 1.8,
                }
            },
        ],
        'earthfloor_centers': [
            { 'area': 5.0, 'name': 'other' },
            { 'area': 5.0, 'name': 'entrance' }
        ],
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 60.0,
  'total_floor_area': 120.0},
 'envelope': {'general_parts': [{'name': 'ceiling_main_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'InputMethod': 'InputUA',
     'UA': 0.24,
     'IsSunshadeInput': False}},
   {'name': 'ceiling_other_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 33.9,
    'space_type': 'other_occupant_room',
    'spec': {'structure': 'wood',
     'InputMethod': 'InputUA',
     'UA': 0.24,
     'IsSunshadeInput': False}},
   {'name': 'ceiling_non_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'non_occupant_room',
    'spec': {'structure': 'wood',
     'InputMethod': 'Inp

In [24]:
d = {
    'common':{
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 90.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'evlp_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 50.85,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 7.7,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_sw',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.47,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_nw',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 22.37,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_ne',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 47.92,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_se',
                'evlp_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'se',
                'area'       : 22.28,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'floor',
                'evlp_type'  : 'floor',
                'next_space' : 'opened_underfloor',
                'direction'  : 'bottom',
                'area'       : 45.05,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 5.27,'is_sunshade_input': False}
            },
            {
                'name'       : 'floor_bath',
                'evlp_type'  : 'floor',
                'next_space' : 'opened_underfloor',
                'direction'  : 'bottom',
                'area'       : 3.31,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 5.27,'is_sunshade_input': False}
            },
        ],
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 22.69,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw', 
                'area'       : 2.38,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne', 
                'area'       : 3.63,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_se',
                'next_space' : 'outdoor',
                'direction'  : 'se', 
                'area'       : 4.37,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            }
        ],
        'doors': [
            {
                'name'       : 'door_nw', 
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 1.89,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'door_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 1.62,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': False
                }
            }
        ],
        'EarthfloorPerimeters': [
            {
                'name'       : 'earthfloor_perimeter_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'length'     : 1.82,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'length'     : 1.37,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_entrance',
                'next_space' : 'open_underfloor',
                'direction'  : 'nw',
                'length'     : 3.19,
                'spec'       : {
                    'psi': 1.8,
                }
            },
        ],
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 30.0,
  'total_floor_area': 90.0},
 'envelope': {'general_parts': [{'name': 'ceiling_main_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'u_value': 7.7,
     'is_sunshade_input': False}},
   {'name': 'ceiling_other_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'other_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'u_value': 7.7,
     'is_sunshade_input': False}},
   {'name': 'ceiling_non_occupant_room',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'non_occupant_room',
